**NOTE:** This was a first implementation and is mainly kept here for legacy purposes.  You should not use code from this file... though the Peer Rank rule from Walsh is implemented correctly.


Better Crowd Sourcing
=========================

Here we are combining bits of Omer's method of impartial peer review with elements of Toby's Markovian/Stable State algorithms.  The basic algorithm implemented below is as follows:

0. We want to select $k$ elements from a set of $n$ agents by partitioning into $m$ agents.
1. Each of the $n$ agents submits a score between $0.0 - 1.0$ for all the other agents.
2. Agents are randomly partitioned into $m$ partitions of size $\frac{n}{m}$.
3. The scores of these partitions are determined by a process (Dividing a Dollar or Markovian) which determines a share for each partition $x_i$.
5. We use an aportionment method to turn the share $x_i$ into integers and select the top elements from that partition.

Biased Peer Rank Method
========================

Below we have implemented hte biased PeerRank method from Toby.  The origional formulation doesn't take into account missing information or what happens when we don't want agents to grade themselves.  We discuss this generalization here.


**In the formulation below $A_{i,j}$ is the grade that agent $j$ gives to agent $i$ This gives us the nice formalism that row i is the grades that agent i has recieved.**

We are given an $n\times n$ matrix where agent $i$ gives a grade to agent $j$ in $A_{j,i}$.  We suppose all marks are in the interval $[0.0, 1.0]$.  We are also given a self-loop constant $\alpha$ in $ 0.0 < \alpha < 1.0$. We are also given a $\beta$ which is the prize for doing well.
Let r_i be the set of agents that $i$ has submitted a grade for.

We must have that $\alpha + \beta \leq 1$. We can now define the start and update steps as:

** Note: fix from here the equations for people raing less than.. only need to modify the second term of the matrix but doesn't matter if beta is 0. **

$$ G^0_i = \frac{1}{|r_i|} \sum_{j=1}^m A_{i,j} $$

and update as:

$$ G^{n+1}_i = (1 - \alpha - \beta) \cdot G^n_i + \frac{\alpha}{|G^{n}|_{\ell=1}} \cdot G^{n} \cdot A_{i,:}^T + \frac{\beta}{m}\sum_{j=0}^{m} 1 - |A_{j,i} - G^n_j| $$

We keep rocking until $ \sum_{i=0}^m |G^{n+1}_i - G^{n}_i|  < 0.00001 $


Dividing A Dollar
=====================


Omer has proposed a divide the dollar scheme where we compute shares for a particular partition $m_i$ as follows.  Let $n - m_i$ be the set of agents outside the partition $m_i$.

$$ m_k = \frac{\sum_{\forall i \in n - m_i, \forall j \in m_i} A_{ji}}{n} $$

This scheme is implemented below.

Note here that we normalize the agents' scores with respect to the outside clusters. So we need to blank them out.


Apportionment
======================
0. Let $s_i$ be the share of $i$ defined as $x_i * k$.
1. Check if $\sum_{i} \lfloor s_i \rfloor = k$ then quit, else. (this implies that all $x_i*k$ are integer and that there could have been no manipulation).
2. Foreach $i$: if $\lceil s_i \rceil - s_i > \frac{1}{n}$ then $s_i = \lfloor s_i \rfloor$ else $s_i = \lceil s_i \rceil$
3. if $\sum_{i} s_i < k $ then increment $k - \sum_{i} s_i$ clusters according to the distribution indiuced by the dollar shares and increase shares of the selected clusters by 1 (with replacement).
4. elif $\sum_{i} s_i > k$ then select $\sum_{i} s_i - k$ according to the inverse dollar share distribution given by $\forall i: \frac{1 - s_i}{\sum_{j}s_j}$ and decrease shares of the selected clusters by 1 (with replacement).

Question: How do we resolve ties in the selection process?


In [11]:
#Load the standard stuff..
import numpy as np
import math
import sys
import itertools
import random
from scipy import stats
import matplotlib.pyplot as plt

from numpy import linalg as LA

# Set DEBUG
DEBUG = True

#Set NUMPY precision.
np.set_printoptions(precision=5, linewidth=200)

#Vector magnitude change.
def delta(c, p):
    return np.sum(np.absolute(c - p))

# Compute biased peer rank
# Recall that we can set B=0 to recover the unbiased version.
def compute_biased_peer_rank(A, alpha, beta):
    #Ensure g is square.
    if A.ndim != 2 or A.shape[0] != A.shape[1]:
        print("Grade matrix is not square")
        return -1

    #num agents:
    m = A.shape[0]
    #Make grade vector of length equal to one leg..
    G = np.ones(m)
    pG = np.zeros(m)
    
    
    #initialize grade vector (X^0) to be 1/m of the matrix -- NOTE: why do I get to grade myself?
    for i in range(m):
        G[i] = 1.0/m * np.sum(A[i, : ])
    #print("Initial G: " + str(G))
    
    #Now we update..
    while delta(G, pG) > 0.00001:
        #print ("Change : " + str(delta(G, pG)))
        pG = G.copy()
        for i in range(m):
            #print("Last Term: " + str([1 - abs(A[j,i] - pG[j]) for j in range(m)]))
            #print(pG)
            G[i] = (1.0 - alpha - beta) * pG[i] + alpha / np.sum(pG) * np.dot(pG, np.vstack(A[i, :])) + beta / m * sum([1 - abs(A[j,i] - pG[j]) for j in range(m)])
        #print("New G: " + str(G))
    
    #print(delta(G, np.zeros(m)))
    #print(delta(np.zeros(m), G))
    #print(np.sum(G))
    
    return(G)

## Use Omer's Divide a Dollar Scheme.
def divide_dollar(A):
    #grades are a vec the same lenght as the column.
    shares = np.zeros(A.shape[1])
    
    #So here the ratio is, for each "partition" the sum of the other 2 partitions divided by n
    for i,j in itertools.permutations(range(A.shape[0]), 2):
        shares[i] += A[i,j]
    return shares
    

## Draw elements according to a discrete distribution.
#Return a value randomly from a given discrete distribution.
def draw(values, distro):
    #This is a bit hacked together -- only need that the distribution
    #sums to 1.0 within 5 digits of rounding.
    if round(sum(distro),5) != 1.0:
        print("Input Distro is not a Distro...")
        print(str(distro) + "  Sum: " + str(sum(distro)))
        exit()
    if len(distro) != len(values):
        print("Values and Distro have different length")

    cv = 0
    draw = random.random() - distro[cv]
    while draw > 0.0:
        cv+= 1
        draw -= distro[cv]
    return values[cv]

    

## Select elements from the p partitions according to the s shares given the a matrix.
def select_winners(p, s, a):
    #For each partition compute the scores of each of the elements
    winners = []
    score_tuples = {}
    for c_p in sorted(p.keys()):
        score_tuples[c_p] = []
        #Add up the grades recieved...
        for i in p[c_p]:
            score_tuples[c_p].append((i, a[i, :].sum()))
        score_tuples[c_p] = sorted(score_tuples[c_p], key=lambda x: x[1], reverse=True)
        # For each partition select the top s elements according to their score.
        # Note that at this point we implicitly deal with ties in lex order.
        winners += score_tuples[c_p][:s[c_p]]
    return winners
        

## Jeffersonian Aportionment..
def jeffersonian_apportionment(k, dist):
    ## Use the jefferson method to apportion this.
    KP = [math.ceil(x * k) - x * k for x in dist]
    KP = sorted(KP)
    kp = 0
    for c in KP:
        kp = c
        if np.floor(dist/dist.sum()*k+kp).sum() == k:
            break
    else:
        print("Didn't find an integer assignment in kp.. big problem")

    print("\nkp and Integer Shares:")
    print("kp = " + str(kp))
    print((k + kp) * dist)
    k_from_partition = (k + kp) * dist
    return k_from_partition

## Randomized Apportionment Described Above.
def randomized_apportionment(n, k, dist):
    #0. Let $s_i$ be the share of $i$ defined as $x_i * k$.
    s = [x*k for x in dist]

    #1. Check if $\sum_{i} \lfloor s_i \rfloor = k$ then quit, else.
    if k != int(sum([math.floor(x) for x in s])):
        #2. Foreach $i$: if $\lceil s_i \rceil - s_i > \frac{1}{n}$ then $s_i = \lfloor s_i \rfloor$ else $s_i = \lceil s_i \rceil$
        s = [int(math.floor(x)) if math.ceil(x) - x > float(1./n) else int(math.ceil(x)) for x in s]
        if DEBUG: print("\nAfter Ceil and Flooring Shares: " + str(s))
            
        #3. if $\sum_{i} s_i < k $ then increment $k - \sum_{i} s_i$ clusters according to the distribution indiuced by the dollar shares and increase shares of the selected clusters by 1 (with replacement).
        if k > int(sum(s)):
            #normalize dist into a distrobution.
            random_dist = [float(x) / dist.sum() for x in dist]
            if DEBUG: print("\nRandomly adding " + str(int(k - sum(s))) + " according to: " + str(random_dist))
            #keep selecting a random number in 0-1.0 and incrementing that share..
            for i in range(int(k - sum(s))):
                s[draw(range(len(s)), random_dist)] += 1
        #4. elif $\sum_{i} s_i > k$ then select $\sum_{i} s_i - k$ according to the inverse dollar share distribution given by $\forall i: \frac{1 - s_i}{\sum_{j}s_j}$ and decrease shares of the selected clusters by 1 (with replacement).
        elif k < int(sum(s)):
            #normalize dist into a distrobution.
            random_dist = [float(x) / dist.sum() for x in dist]
            random_dist = [1. - x for x in random_dist]
            norm_factor = random_dist.sum()
            random_dist = [x / norm_factor for x in random_dist]
            print("Randomly removing " + str(int(sum(s) - k)) + " according to: " + str(random_dist))
            #keep selecting a random number in 0-1.0 and incrementing that share..
            for i in range(int(sum(s)) - k):
                s[draw(range(len(s)), random_dist)] += 1
    else:
        s = [int(math.floor(x)) for x in s]
    
    if DEBUG: print("\nFinal Apportionment is: " + str(s))
    return s
    
    

    

    

    
### Prize Selection -- 
def Impartial_Prize_Selection(A, k, m, shuffle=True):
    #Sanity check..
    if A.ndim != 2 or A.shape[0] != A.shape[1]:
        print("Grade matrix is not square")
        return 0
    ## Set N for convinence...
    n = A.shape[0]
    
    #Enforce the Diagonal is 0's
    for i in range(A.shape[0]):
        A[i,i] = 0.
    
    if DEBUG: print("Grade Matrix:\n"+str(A))

    index_set = list(range(n))
    if shuffle:
        random.shuffle(index_set)

    #Create partition index sets..
    partitions = {}
    #evenly distributed.
    p_elements = int(math.ceil(float(n)/m))

    for i in range(m):
        if len(index_set) >= p_elements:
            partitions[i] = index_set[:p_elements]
            index_set = index_set[p_elements:]
        else:
            partitions[i] = index_set

    if DEBUG: print("\nPartitions:\n" + str(partitions))


    # Note we have to blank out the scores of the other agents in our partitions
    # before we do anything else.  We know we already have a 0. grade for ourselves.
    for c in partitions.values():
        for pair in itertools.permutations(c, 2):
            A[pair[0],pair[1]] = 0.

    #Normalize so that each Column sums to 1!
    col_sums = A.sum(axis=0)
    norm_A = A / col_sums[numpy.newaxis , : ]

    if DEBUG: print("\nNormalized Grade Matrix:\n" + str(norm_A))

    #Create and Conglomerate the partition scores..
    p_scores = zeros((m,m))

    #Compute the score that partition pair[1] gives to pair[0]..
    for pair in itertools.permutations(list(range(m)), 2):
        t = 0.0
        for i in partitions[pair[0]]:
            for j in partitions[pair[1]]:
                t += norm_A[i,j]
        p_scores[pair[0], pair[1]] = t

    if DEBUG: print("\nGroup Scores:\n" + str(p_scores))
    
    # Compute Shares based on the dividing a dollar.
    dist = divide_dollar(p_scores)
    dist = dist / float(n)
    if DEBUG: print("\nDistribution:\n" + str(dist))

    shares = randomized_apportionment(n, k, dist)
    if DEBUG: print("\nFinal Apportionment:\n" + str(shares))

    ## For each partition, return the top selections from the partition.
    winning_set = select_winners(partitions, shares, norm_A)
    if DEBUG:
        print("\nWinners:")
        for agent,score in winning_set:
            print("Agent: " + str(agent) + " with Score: " + str(score))

    return winning_set


In [12]:
A = np.array([[0.0 , .9  , .9 , .9, .9, .9, .9, .9, .9 ],
              [.5  , 0.0 , .50, .9, .9, .9, .9, .9, .9 ],
              [.75 , .50 , 0.0, .9, .9, .9, .9, .9, .9 ],
              [.90 , .9  , .9 , 0.0, .9, .9, .9, .9, .9 ],
              [.5  , .90 , .50, .9, 0.0, .9, .9, .9, .9 ],
              [.75 , .50 , 1.0, .9, .9, .9, .9, .9, .9 ],
              [.90 , .9  , .9 , .9, .9, .9, .9, .9, .9 ],
              [.5  , .90 , .50, .9, .9, .9, .9, .9, .9 ],
              [.75 , .50 , 1.0, .9, .9, .9, .9, .9, .9 ]])

Impartial_Prize_Selection(A, 4, 3, shuffle=True)


Grade Matrix:
[[ 0.    0.9   0.9   0.9   0.9   0.9   0.9   0.9   0.9 ]
 [ 0.5   0.    0.5   0.9   0.9   0.9   0.9   0.9   0.9 ]
 [ 0.75  0.5   0.    0.9   0.9   0.9   0.9   0.9   0.9 ]
 [ 0.9   0.9   0.9   0.    0.9   0.9   0.9   0.9   0.9 ]
 [ 0.5   0.9   0.5   0.9   0.    0.9   0.9   0.9   0.9 ]
 [ 0.75  0.5   1.    0.9   0.9   0.    0.9   0.9   0.9 ]
 [ 0.9   0.9   0.9   0.9   0.9   0.9   0.    0.9   0.9 ]
 [ 0.5   0.9   0.5   0.9   0.9   0.9   0.9   0.    0.9 ]
 [ 0.75  0.5   1.    0.9   0.9   0.9   0.9   0.9   0.  ]]

Partitions:
{0: [5, 3, 4], 1: [8, 2, 0], 2: [1, 7, 6]}

Normalized Grade Matrix:
[[ 0.       0.21429  0.       0.16667  0.16667  0.16667  0.16667  0.16667  0.     ]
 [ 0.12346  0.       0.11628  0.16667  0.16667  0.16667  0.       0.       0.16667]
 [ 0.       0.11905  0.       0.16667  0.16667  0.16667  0.16667  0.16667  0.     ]
 [ 0.22222  0.21429  0.2093   0.       0.       0.       0.16667  0.16667  0.16667]
 [ 0.12346  0.21429  0.11628  0.       0.       0.    

[(3, 1.1458102620893318),
 (0, 1.0476190476190477),
 (6, 1.0981912144702841),
 (1, 0.90640252655756526)]